# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [4]:
%%capture
%pip install tensorflow_io sagemaker -U

In [5]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [6]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::814648165615:role/service-role/AmazonSageMaker-ExecutionRole-20230402T204258


In [7]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://shenhapy/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [8]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [9]:
# # build and push the docker image. This code can be commented after being ran once.
# # This will take around 10 mins.
# image_name = 'tf2-object-detection'
# !sh ./docker/build_and_push.sh $image_name

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [10]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

814648165615.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20230402185236


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [11]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d1_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.index


mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-04-02 22:06:47--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.63.128, 2607:f8b0:4004:c1b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.63.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘/tmp/efficientdet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 11.0M 4s
    50K .......... .......... .......... .......... ..........  0% 20.5M 3s
   100K .......... .......... .......... .......... ..........  0% 21.0M 3s
   150K .......... .......... .......... .......... ..........  0% 21.7M 3s
   200K .......... .......... .......... .......... ..........  0% 20.7M 3s
   250K .......... .......... .......... .......... ..........  0% 65.5M 2s
   300K ..........

## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [12]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    source_dir='source_dir/',
    entry_point='run_training.sh',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.trn1.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-04-02-22-06-48-480


2023-04-02 22:06:51 Starting - Starting the training job...
2023-04-02 22:07:05 Starting - Preparing the instances for training...
2023-04-02 22:07:49 Downloading - Downloading input data...
2023-04-02 22:08:10 Training - Downloading the training image.........
2023-04-02 22:10:01 Training - Training image download completed. Training in progress.....2023-04-02 22:10:29,810 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-02 22:10:29,812 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-02 22:10:29,824 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-02 22:10:29,826 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-02 22:10:29,836 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-02 22:10:29,838 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neuron

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0402 22:10:42.900502 140220113835840 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0402 22:10:46.113400 140220113835840 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0402 22:10:48.000422 140220113835840 module_wrapper.py:149] From /usr/local/lib/python3.8/dist-packages/object_detection/builders/optimizer_builder.py:124: The name tf.keras.optimizers.SGD is deprecated

INFO:tensorflow:Step 200 per-step time 3.488s
I0402 22:23:45.105939 140220113835840 model_lib_v2.py:705] Step 200 per-step time 3.488s
INFO:tensorflow:{'Loss/classification_loss': 0.3837895,
 'Loss/localization_loss': 0.037448704,
 'Loss/regularization_loss': 0.029545741,
 'Loss/total_loss': 0.45078397,
 'learning_rate': 0.0073200003}
I0402 22:23:45.106245 140220113835840 model_lib_v2.py:708] {'Loss/classification_loss': 0.3837895,
 'Loss/localization_loss': 0.037448704,
 'Loss/regularization_loss': 0.029545741,
 'Loss/total_loss': 0.45078397,
 'learning_rate': 0.0073200003}
INFO:tensorflow:Step 300 per-step time 3.487s
I0402 22:29:33.834827 140220113835840 model_lib_v2.py:705] Step 300 per-step time 3.487s
INFO:tensorflow:{'Loss/classification_loss': 0.2756325,
 'Loss/localization_loss': 0.019653795,
 'Loss/regularization_loss': 0.029549213,
 'Loss/total_loss': 0.3248355,
 'learning_rate': 0.010480001}
I0402 22:29:33.835088 140220113835840 model_lib_v2.py:708] {'Loss/classification_lo

INFO:tensorflow:Step 1600 per-step time 3.488s
I0402 23:45:10.970550 140220113835840 model_lib_v2.py:705] Step 1600 per-step time 3.488s
INFO:tensorflow:{'Loss/classification_loss': 0.3340661,
 'Loss/localization_loss': 0.013385174,
 'Loss/regularization_loss': 0.030127086,
 'Loss/total_loss': 0.37757835,
 'learning_rate': 0.05156}
I0402 23:45:10.970839 140220113835840 model_lib_v2.py:708] {'Loss/classification_loss': 0.3340661,
 'Loss/localization_loss': 0.013385174,
 'Loss/regularization_loss': 0.030127086,
 'Loss/total_loss': 0.37757835,
 'learning_rate': 0.05156}
INFO:tensorflow:Step 1700 per-step time 3.490s
I0402 23:50:59.933722 140220113835840 model_lib_v2.py:705] Step 1700 per-step time 3.490s
INFO:tensorflow:{'Loss/classification_loss': 0.25040656,
 'Loss/localization_loss': 0.011024523,
 'Loss/regularization_loss': 0.030205987,
 'Loss/total_loss': 0.2916371,
 'learning_rate': 0.05472}
I0402 23:50:59.933974 140220113835840 model_lib_v2.py:708] {'Loss/classification_loss': 0.25

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0403 00:08:38.599064 140236357375808 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0403 00:08:39.737470 140236357375808 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0403 00:08:42.000599 140236357375808 module_wrapper.py:149] From /usr/local/lib/python3.8/dist-packages/object_detection/builders/optimizer_builder.py:124: The name tf.keras.optimizers.SGD is deprecated

INFO:tensorflow:Waiting for new checkpoint at /opt/training
I0403 00:13:42.101101 140236357375808 checkpoint_utils.py:168] Waiting for new checkpoint at /opt/training
INFO:tensorflow:Timed-out waiting for a checkpoint.
I0403 00:13:51.113124 140236357375808 checkpoint_utils.py:231] Timed-out waiting for a checkpoint.
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=9.89s).
Accumulating evaluation results...
DONE (t=0.21s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.178
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.060
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.262
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.379
 Aver

W0403 00:15:03.202756 140392352532288 save.py:274] Found untraced functions such as WeightSharedConvolutionalBoxPredictor_layer_call_fn, WeightSharedConvolutionalBoxPredictor_layer_call_and_return_conditional_losses, WeightSharedConvolutionalBoxHead_layer_call_fn, WeightSharedConvolutionalBoxHead_layer_call_and_return_conditional_losses, WeightSharedConvolutionalClassHead_layer_call_fn while saving (showing 5 of 535). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /tmp/exported/saved_model/assets
I0403 00:15:31.495609 140392352532288 builder_impl.py:797] Assets written to: /tmp/exported/saved_model/assets
INFO:tensorflow:Writing pipeline config file to /tmp/exported/pipeline.config
I0403 00:15:34.812107 140392352532288 config_util.py:253] Writing pipeline config file to /tmp/exported/pipeline.config
2023-04-03 00:15:37,438 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-04-03 00:15:50 Uploading - Uploading generated 

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [13]:
# your writeup goes here.